Firstly, the authentication is done.

In [14]:
import ee
import geemap

In [15]:
# # Trigger the authentication flow.
# ee.Authenticate()

In [16]:
try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


Getting data and vizualizing

Stockholm, Sweden is selected as our region of interest and center the base map on this city.

In [17]:
# Stockholm 

point = ee.Geometry.Point([18, 59.3])
lon, lat = point.getInfo()['coordinates']

 

The first interactive map is created with one line of code, specifying the location where we want to center the map, the zoom level, and the main dimensions of the map:

In [18]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Next, Sentinel 2 multispectral images of Stockholm are gathered with less than 10 % could coverage during the summer 2023, and let's take the median of all these images to have one single almost cloud free image.

In [19]:
image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(point) \
    .filterDate('2023-06-21', '2023-09-21') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10) \
    .sort('CLOUD_COVER')\
    .first() 

In [20]:
# Set visualization parameters for land cover.
vis_params = {
    'min': 0,'max': 4000,
    'bands': ["B4", "B3", "B2"]
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, 'S3-2023-Median')

In [21]:
Map

Map(bottom=19584.0, center=[59.300000000000004, 18], controls=(WidgetControl(options=['position', 'transparent…

## Check image properties

In [22]:
props = geemap.image_props(image)
props.getInfo()

{'AOT_RETRIEVAL_ACCURACY': 0,
 'AOT_RETRIEVAL_METHOD': 'SEN2COR_DDV',
 'BOA_ADD_OFFSET_B1': -1000,
 'BOA_ADD_OFFSET_B10': -1000,
 'BOA_ADD_OFFSET_B11': -1000,
 'BOA_ADD_OFFSET_B12': -1000,
 'BOA_ADD_OFFSET_B2': -1000,
 'BOA_ADD_OFFSET_B3': -1000,
 'BOA_ADD_OFFSET_B4': -1000,
 'BOA_ADD_OFFSET_B5': -1000,
 'BOA_ADD_OFFSET_B6': -1000,
 'BOA_ADD_OFFSET_B7': -1000,
 'BOA_ADD_OFFSET_B8': -1000,
 'BOA_ADD_OFFSET_B8A': -1000,
 'BOA_ADD_OFFSET_B9': -1000,
 'CLOUDY_PIXEL_OVER_LAND_PERCENTAGE': 6.55647,
 'CLOUDY_PIXEL_PERCENTAGE': 4.970358,
 'CLOUD_COVERAGE_ASSESSMENT': 4.970358,
 'CLOUD_SHADOW_PERCENTAGE': 4.098606,
 'DARK_FEATURES_PERCENTAGE': 0.035381,
 'DATASTRIP_ID': 'S2A_OPER_MSI_L2A_DS_2APS_20230622T162904_S20230622T100756_N05.09',
 'DATATAKE_IDENTIFIER': 'GS2A_20230622T100601_041775_N05.09',
 'DATATAKE_TYPE': 'INS-NOBS',
 'DEGRADED_MSI_DATA_PERCENTAGE': 0.0001,
 'FORMAT_CORRECTNESS': 'PASSED',
 'GENERAL_QUALITY': 'PASSED',
 'GENERATION_TIME': 1687451344000,
 'GEOMETRIC_QUALITY': 'PASSED',

## Make a training dataset

In [23]:
training = image.sample(**{
    'scale': 30,
    'numPixels': 4015,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(training, {}, 'training', False)
Map

Map(bottom=19584.0, center=[59.300000000000004, 18], controls=(WidgetControl(options=['position', 'transparent…

## Clustering

We first train a k-means clustering algorithm and then use it to classify the whole image. The number of clusters is defined 5 as there are 5 land covers.

In [24]:
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

In [25]:
# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=19584.0, center=[59.300000000000004, 18], controls=(WidgetControl(options=['position', 'transparent…